In [1]:
import os
from pathlib import Path 
import numpy as np
import nibabel as nib
import nibabel.processing
import time
import sys
from scipy import sparse
from matplotlib import pyplot as plt
from nilearn import plotting, image
from scipy import ndimage
import argparse 
import tempfile
import subprocess
import pickle
import shutil
from itertools import repeat
from scipy.sparse import csr_matrix

In [23]:
chaco_root = Path("/mnt/h/srs-9/chaco")
scanID = "sub1001-20170215"
chaco_jobID = "lesion_mask_mni_nemo_output_ifod2act_20260222_203419351"
scan_root = chaco_root / scanID
chaco_outdir = scan_root / chaco_jobID
method = "ifod2act"
output_basename = "lesion_mask_mni_nemo_output" + "_" + method

In [18]:
def load_pkl(filesuffix):
    filepath = chaco_outdir / f"{output_basename}_{filesuffix}"
    return pickle.load(open(filepath,"rb"))

def load_nib_data(filesuffix):
    filepath = chaco_outdir / f"{output_basename}_{filesuffix}"
    return nib.load(filepath).get_fdata()

def flatParcellationToTransform(Pflat, isubj=None, out_type="csr", max_sequential_roi_value=None):
    if sparse.issparse(Pflat):
        Pdata=Pflat[isubj,:].toarray().flatten()
    elif isubj is None:
        Pdata=Pflat.flatten()
    elif len(Pdata.shape)==2:
        Pdata=Pflat[isubj,:].flatten()
            
    numvoxels=np.prod(Pdata.shape)
    pmaskidx=np.where(Pdata!=0)[0]
    uroi, uidx=np.unique(Pdata[Pdata!=0],return_inverse=True)
    numroi=len(uroi)
    
    if max_sequential_roi_value is not None:
        #this would create an entry at the actual ROI values, rather than just going through the sequential PRESENT value
        #eg: for cc400 it would be a 7M x 400 array instead of 7M x 392
        #   but for an arbitrary/custom input, where they left freesurfer values, this could make it in the thousands!
        uidx=(uroi[uidx]-1).astype(np.int64)
        numroi=max_sequential_roi_value.astype(np.int64)
    
    if out_type == "csr":
        return sparse.csr_matrix((np.ones(pmaskidx.size),(pmaskidx,uidx)),shape=(numvoxels,numroi),dtype=np.float32)
    elif out_type == "csc":
        return sparse.csc_matrix((np.ones(pmaskidx.size),(pmaskidx,uidx)),shape=(numvoxels,numroi),dtype=np.float32)


In [20]:
thomas_filepath = scan_root / "thomas_thalamus.nii.gz"
Pdata = nib.load(thomas_filepath).get_fdata()
Psparse = flatParcellationToTransform(Pdata.flatten(), None, out_type="csr", max_sequential_roi_value=None)

In [57]:
suffix = "chacovol_res1mm_allref.pkl"

thal_allref_suffix = "chacovol_thomas_thalamus_allref.pkl"
thal_allref = load_pkl(thal_allref_suffix)
print(thal_allref.shape, type(thal_allref))

thal_allref_suffix = "chacovol_thomas_thalamus_allref_denom.pkl"
thal_allref_denom = load_pkl(thal_allref_suffix)
print(thal_allref_denom.shape)

thal_mean_suffix = "chacovol_thomas_thalamus_mean.pkl"
thal_mean = load_pkl(thal_mean_suffix)
print(thal_mean.shape, type(thal_mean))

(420, 10) <class 'scipy.sparse._csr.csr_matrix'>
(420, 10)
(1, 10) <class 'numpy.ndarray'>


/tmp/ipykernel_751328/1536509588.py:3: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(open(filepath,"rb"))


In [54]:
print([float(item) for item in thal_mean[0]])

[0.040872424840927124, 0.030032090842723846, 0.010976091958582401, 0.019914278760552406, 0.027912579476833344, 0.05090194568037987, 0.029394857585430145, 0.03634316474199295, 0.022196678444743156, 0.03861214593052864]


In [59]:
thal_mean[0]

array([0.04087242, 0.03003209, 0.01097609, 0.01991428, 0.02791258,
       0.05090195, 0.02939486, 0.03634316, 0.02219668, 0.03861215],
      dtype=float32)

In [58]:
check = thal_allref.mean(axis=0)
check

matrix([[0.04087242, 0.03003209, 0.01097609, 0.01991428, 0.02791258,
         0.05090195, 0.02939486, 0.03634316, 0.02219668, 0.03861215]],
       dtype=float32)

In [15]:
thal_mean

array([[0.04060829, 0.02978908, 0.0110036 , 0.02208218, 0.02835336,
        0.0338433 , 0.02986868, 0.03631852, 0.02170792, 0.03713765]],
      dtype=float32)

In [34]:
suffix = "chacoconn_thomas_thalamus_allref.pkl"
chacoconn_thomas = load_pkl(suffix)
type(chacoconn_thomas[0])

/tmp/ipykernel_751328/1536509588.py:3: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  return pickle.load(open(filepath,"rb"))


scipy.sparse._csr.csr_matrix

In [42]:
suffix = "chacovol_res1mm_mean.nii.gz"
chacovol_mean = load_nib_data(suffix)
suffix = "chacovol_res1mm_stdev.nii.gz"
chacovol_std = load_nib_data(suffix)

mean_chacos1 = []
mean_chacos2 = []
for idx in range(1, 11):
    chaco_means = chacovol_mean[Pdata == idx]
    mean_chacos1.append(chaco_means.mean())
    chaco_stds = chacovol_std[Pdata == idx]
    weighted_chaco = np.sum(chaco_means / chaco_stds**2) / np.sum(1 / chaco_stds**2)
    mean_chacos2.append(weighted_chaco)

/tmp/ipykernel_751328/2055686161.py:12: RuntimeWarning: invalid value encountered in divide
  weighted_chaco = np.sum(chaco_means / chaco_stds**2) / np.sum(1 / chaco_stds**2)
/tmp/ipykernel_751328/2055686161.py:12: RuntimeWarning: divide by zero encountered in divide
  weighted_chaco = np.sum(chaco_means / chaco_stds**2) / np.sum(1 / chaco_stds**2)


In [46]:
print([float(item) for item in mean_chacos1])

[0.056728043221417314, 0.07048230594109417, 0.04970836273381652, 0.05833122023618951, 0.03717169936724419, 0.06497520823226782, 0.009711612824216181, 0.04217460730063749, 0.05985130600734626, 0.07280325995909334]


In [41]:
print(chacovol_mean.shape, type(chacovol_mean))

(182, 218, 182) <class 'numpy.ndarray'>


In [55]:
import pickle
import numpy as np

numer = pickle.load(open(chaco_outdir / f"{output_basename}_chacovol_thomas_thalamus_allref.pkl", "rb"))      # (420, 10)
denom = pickle.load(open(chaco_outdir / f"{output_basename}_chacovol_thomas_thalamus_allref_denom.pkl", "rb")) # (420, 10)

# convert sparse to dense if needed
numer = np.array(numer.todense()) if hasattr(numer, 'todense') else np.array(numer)
denom = np.array(denom.todense()) if hasattr(denom, 'todense') else np.array(denom)

# per-subject ratio, then mean across subjects
ratio_per_subj = np.divide(numer, denom, out=np.zeros_like(numer), where=denom!=0)
roi_chaco_mean = ratio_per_subj.mean(axis=0)  # shape (10,)

/tmp/ipykernel_751328/234137148.py:4: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  numer = pickle.load(open(chaco_outdir / f"{output_basename}_chacovol_thomas_thalamus_allref.pkl", "rb"))      # (420, 10)
/tmp/ipykernel_751328/234137148.py:5: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  denom = pickle.load(open(chaco_outdir / f"{output_basename}_chacovol_thomas_thalamus_allref_denom.pkl", "rb")) # (420, 10)


In [56]:
roi_chaco_mean

array([6.1306814e-03, 8.5499551e-04, 3.1635392e-04, 7.1460097e-05,
       1.7609031e-04, 1.1420832e-04, 5.5174460e-04, 8.5331366e-04,
       3.5964456e-04, 4.0161208e-04], dtype=float32)